# Athena

## What is Athena?

>[Amazon Athena](https://docs.aws.amazon.com/athena/latest/ug/what-is.html) is an interactive query service that makes it easy to analyze data directly in Amazon Simple Storage Service (Amazon S3) using standard SQL.

### Advantages

- Serverless = easy to set up and use.
- Fast.
- You pay only for the queries you run.
- Works with standard data formats like CSV, JSON, ORC, Avro and Parquet.
- It is integrated with [AWS Glue](https://aws.amazon.com/glue/) (i.e. AWS ETL tool).

### Disadvantages

- Some queries with extremely large data might be nearly impossible to run (unless you find some hack).
- It does not handle sorting large datasets well.
- OpenCSV SerDe has troubles with NULL values in columns other than strings.
- Data needs to be stored on S3, so you additionally pay for it's storage.
- Might be costly (depending on the usage).

### Pricing

- You are charged for the number of bytes scanned by Athena by every query you execute, with a 10MB minimum per query. 
TB of data scanned costs $5. 
- There are no charges for Data Definition Language (DDL) statements like CREATE/ALTER/DROP TABLE, statements for managing partitions, or failed queries.
- Cancelled queries are charged based on the amount of data scanned.
- When calculating the whole cost of the service it is worth remembering that you need to include the cost of S3 data storage.


### How does Athena work?

Athena uses two components:
- [Presto](https://prestodb.io/docs/0.172/) as a distributed SQL engine to run queries, and 
- [Apache Hive](https://en.wikipedia.org/wiki/Apache_Hive) as [DDL](https://en.wikipedia.org/wiki/Data_definition_language) (to create, drop, and alter tables and partitions). 

## Let's check Athena out!
Before we try Athena out, we need to have some data to work with on S3.

### Exercise 0 

Upload titanic.csv file from your data folder to the separate bucket on s3. Remember that you need to give your bucket a unique name. Note somewhere the path to the bucket, as we will need it later.

Once we have our data stored on S3 we can create a table out of it in Athena. To achieve this we need to do the following:

1. Open Athena service in AWS.

<img src='img/athena.png' width=900>


2. Click "Create table", and then "from S3 bucket data".

<img src='img/create_table.png' width=900>


3. Fill in Step 1 with the following:
    - Database = titanic_db
    - Table Name = titanic
    - Location of Input Data Set = location of your csv file ex. s3://titanic-data-2019-01/ (you need to put your's bucket name here)
    
    and click "Next".
   
   
4. In Step 2 select "CSV".
5. In Step 3 click on "Bulk add columns" and paste there the following:

    ```
    passengerid int, survived int, pclass smallint, name string, sex string, age double, 
    sibsp int, parch int, ticket string, fare double,cabin string, embarked string 
    ```
   </br>
   and then click "Next".
   
   
6. In Step 4 click on "Create table".
7. If you see a new table in the Tables section and you get "Query successful." message in Results section of the console it means that you did it!

<img src='img/success.png' width=900>



Note that Athena translates what we did in the wizard into a query, that we could have used instead of using the wizard. The query looks like that:


```
CREATE EXTERNAL TABLE IF NOT EXISTS titanic_db.titanic (
  `passengerid` int,
  `survived` int,
  `pclass` smallint,
  `name` string,
  `sex` string,
  `age` double,
  `sibsp` int,
  `parch` int,
  `ticket` string,
  `fare` double,
  `cabin` string,
  `embarked` string 
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
) LOCATION 's3://titanic-data-2019-01/' -- you need to put your bucket name here
TBLPROPERTIES ('has_encrypted_data'='false');
```

You can check it out by deleting the table that we have created either by clicking on the three dots next to the table name and then clicking on 'Delete table' or by executing `DROP TABLE titanic_db.titanic;`. Then paste `CRATE EXTERNAL TABLE...` query to the console and execute it. It should create a new table with Titanic's data as a result.

As we have mentioned previously Athena uses Presto as an SQL engine. It's syntax is similar to other SQL dialects. Let's check how our data looks like with a simple query:

```
SELECT *
FROM titanic_db.titanic
LIMIT 10
```

Ups, it turns out our data is not correctly parsed into columns. In our `CREATE EXTERNAL TABLE` query the element that is responsible for parsing is called SerDe. LazySimpleSerDe is default one but as it turns out it does not handle double quotes. We need to use different SerDe.

## Exercise 1 (5 min)

Check [Athena's documentation](https://docs.aws.amazon.com/athena/latest/ug/csv.html) and try to modify `CREATE EXTERNAL TABLE` query so it parses our data correctly. You can write your query here:

-- 

your query


--

## Advanced Functions in Athena

Let's play with our data a bit. Presto has an extensive [library of functions](https://prestodb.io/docs/0.172/functions.html) that allows us to do quite sophisticated manipulations with our data. We will look at a few examples in this section.

### Regular Expression Functions

There are [several functions](https://prestodb.io/docs/0.172/functions/regexp.html) in Presto for using regular expressions. Let's assume that we would like to extract a title from the name of every person. We can do this with the following query:

```
SELECT name, regexp_replace(regexp_extract(name, '[A-Za-z]+\.'), '\.') title
FROM titanic_db.titanic
```

### Exercise 2 (5 min)

Create a query that will, based a on the `cabin` column, create three new columns:
- last_cabin: contains code for the last cabin in the sequence (ex. cabin = 'C23 C25 C27', then last_cabin = 'C27'). In case there is only one cabin code, show that code.
- last_cabin_letter: contains just a letter from the last_cabin (ex. first_cabin = 'C23', then last_cabin_letter = 'C').
- last_cabin_number: contains number(s) from the last_cabin (ex. first_cabin = 'C23', then last_cabin_number = '23').

You can write your query here:

-- 

your query


--

### Window functions

Another useful group of functions, both in Presto and in other SQL dialects, are [window functions](https://prestodb.io/docs/0.172/functions/window.html). They allow us to calculate aggregate statistics on specific groups for every row of the data, without reducing the number of rows.

In general, window functions consist of 3 elements:
- a function that will be applied to a specific column,
- partition specification, separating rows into different groups (so it works similarily to the GROUP BY statement),
- ordering specification, which determines order in which input rows will be processed by the window function

So the template for the most window functions looks like that:
`function_name(column_name) OVER(PARTITION BY column_name, ... [ORDER BY column_name, ...])`


As an example, let's say we want to calculate the difference between the age of every person and the mean age for a given sex. Without a window function we could do this in the following way:

```
SELECT 
    t.name, 
    t.sex, 
    age, 
    a.avg_age, 
    CAST(t.age as DOUBLE) - a.avg_age age_diff_avg
FROM titanic_db.titanic t
    JOIN (
        SELECT sex, avg(CAST(age AS DOUBLE)) avg_age
        FROM titanic_db.titanic
        WHERE age != ''
        GROUP BY sex) a ON a.sex = t.sex
WHERE t.age != ''
```

If we decide to use a window function, we can simplify this query to:

```
SELECT 
    name, 
    sex, 
    age, 
    AVG(CAST(age as DOUBLE)) OVER (PARTITION BY sex) avg_age, 
    CAST(age AS DOUBLE) - AVG(CAST(age as DOUBLE)) OVER (PARTITION BY sex)
FROM titanic_db.titanic
WHERE age != ''
```

### Exercise 3 (5 min)
Assuming the price of ticket should reflect port of embarkation (`embarked`) and class (`class`), write a query that will show the number of passangers that overpaid for their tickets (i.e. paid more than average for their port of embarkation and class). You can write your query here:

-- 

your query


--

### Array functions

Presto has also pretty useful set of [functions](https://prestodb.io/docs/0.172/functions/array.html) to work with arrays.

Let's say that we would like to put our data into more concise form, and instead of having a row for every passanger, we would like to have a cabin in every row and in columns data concerning passangers of those cabins. We could do it with arrays:
```
SELECT 
    cabin, 
    array_agg(name) names,
    array_agg(sex) sexes,
    array_agg(CAST(age as double)) ages
FROM titanic_db.titanic
WHERE age != '' and cabin != ''
GROUP BY cabin
```

We can then do some calculations on those arrays like calculate the age of the oldest person that was travelling in each cabin:
```
SELECT cabin, ages, array_max(ages) max_age
FROM (
    SELECT 
        cabin, 
        array_agg(name) names,
        array_agg(sex) sexes,
        array_agg(CAST(age as double)) ages
    FROM titanic_db.titanic
    WHERE age != '' and cabin != ''
    GROUP BY cabin)
```

### Exercise 4 (5 min)

For every cabin count how many underage passangers were occupying it. 

You can write your query here:

-- 

your query


--

## Creating tables in Athena

In [October 2018](https://docs.aws.amazon.com/athena/latest/ug/release-note-2018-10-10.html) Amazon introduced a feature to create a new table based on the result of another query. Before that, creating tables involved a lot of hacks, but now it is pretty simple. We just need to use `CREATE TABLE table_name AS query` statement and a new table with results of the query will be created for us:
```
CREATE TABLE titanic_females AS 
SELECT * 
FROM titanic_db.titanic 
WHERE sex = 'female'
```
Data is by default in the [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) format in the location: 

`s3://aws-athena-query-results-<account>-<region>/<query-name-or-unsaved>/<year>/<month/<date>/<query-id>/` 

However, it would be prudent to change this default location, to organise your data in more reasonable manner (e.g. separate bucket/prefix for every table). A query result location can be changed in Athena settings. Keep in mind that you need to change it every time you run a query, to not to populate your table with unwanted data. As you can imagine it is quite burdersome, so it might be better to do it programatically.

## Working with Athena from Python

### Getting query results into Python

[Pyathena](https://pypi.org/project/PyAthena/) is the package that works well with Athena and is quite easy to use. It has a `connect` function in which you can specify security credentials, staging dir (i.e. where results of your query will be saved), and region that you are set up in. If you don't specify security credentials it will look for default credentials in `.aws/credentials` on your machine (for Unix systems).

In [2]:
import pandas as pd
from pyathena import connect

conn = connect(s3_staging_dir='s3://aws-athena-query-results-920137764546-us-east-1/', # put default query folder here
               region_name='us-east-1')

`connect` function creates an instance of `pyathena.connection.Connection` object.

In [3]:
conn

We can then use this object, for example in pandas `read_sql` function, to download results of a query as pandas DataFrame.

In [5]:
df = pd.read_sql("SELECT * FROM titanic_db.titanic LIMIT 10", conn)
df.head(10)

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,,S
5,6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,,C


The above method is ok for smaller data, but it usually takes quite a long time to download larger data sets. In those instances it usually better to execute query and to download the resulting csv file directly from s3. We can use the same connect object to do this:

In [16]:
cursor = connect(s3_staging_dir='s3://titanic-females/', # insert your bucket here
                 region_name='us-east-1').cursor()

cursor.execute("SELECT * FROM titanic_db.titanic WHERE sex = 'female'")

In [17]:
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('titanic-females')
[bucket.download_file(o.key, 'data/' + o.key) for o in bucket.objects.all() 
 if 'csv' in o.key and 'metadata' not in o.key]

[None]

Alternatively, we could use boto3's Athena client, which has a `start_query_execution` method that executes query and returns metada of the query:

In [18]:
import boto3

client = boto3.client('athena')
query = "SELECT * FROM titanic_db.titanic WHERE sex = 'female'"
response = client.start_query_execution(
    QueryString=query,
    ResultConfiguration={
        'OutputLocation': 's3://titanic-females/', # insert your bucket here
        'EncryptionConfiguration': {
            'EncryptionOption': 'SSE_S3',
        }
    })

This metadata contains, among others, the QueryExecutionId which is also the name of the csv file that we are interested in.

In [19]:
response

{'QueryExecutionId': '35807c98-f98f-41b9-91b5-d2d8f065b73f',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '59',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 16 Jan 2019 13:18:10 GMT',
   'x-amzn-requestid': '65afe49a-7be1-43de-8a9b-30047424705b'},
  'HTTPStatusCode': 200,
  'RequestId': '65afe49a-7be1-43de-8a9b-30047424705b',
  'RetryAttempts': 0}}

We can then use this id to download the csv file with the result directly:

In [20]:
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('titanic-females')
bucket.download_file(response['QueryExecutionId']+'.csv', 'data/query_result.csv')

### Creating tables using Python
We can create tables directly from Python using `CREATE TABLE AS` queries and the same `cursor.execute` method:

In [21]:
cursor = connect(s3_staging_dir='s3://titanic-females-table/', # insert your bucket here
                 region_name='us-east-1').cursor()
create_tbl_query = "CREATE TABLE titanic_females AS SELECT * FROM titanic_db.titanic WHERE sex = 'female'"
cursor.execute(create_tbl_query)

The code above has created a table in default schema. To create it in `titanic_db` schema we can again use `start_query_execution` with QueryExecutionContext set to the proper schema:

In [15]:
create_tbl_resp = client.start_query_execution(
    QueryString=create_tbl_query,
    QueryExecutionContext={
        'Database': 'titanic_db'
    },
    ResultConfiguration={
        'OutputLocation': 's3://titanic-females-table/', # insert your bucket here
        'EncryptionConfiguration': {
            'EncryptionOption': 'SSE_S3',
        }
    })

## Summary

We have only scratched the surface here, but a few things to keep in mind are:
- Athena is AWS tool for analysing data stored on S3 using SQL syntax.
- It uses Presto as na SQL engine, which is quite capable.
- You can work with Athena using AWS console, Python or data base tools like DBeaver.